### Evaluating CPM with CEBaB
Our Causal Proxy Model (CPM) is for providing concept-based explanation for a blackbox model. We use newly developed CEBaB benchmark for comparing CPM with other concept-based explanation methods. This notebook evaluates CPM with CEBaB benchmark under different settings.

More importantly, we introduce new baselines for CPM as well. Formally, we evaluate the blackbox model with interchange intervention evaluation (which will be introduced in details below).

In this notebook, we can evaluate the following models:
- CPM: `BERT-base-uncased`
- CPM: `RoBERTa-base`
- CPM: `GPT2`
- CPM: `LSTM+GloVe`
- CPM: `Control`

and we can evaluate with the following conditions:
- 2-class
- 3-class
- 5-class

#### Imports and Libs

In [10]:
from libs import *
from modelings.modelings_bert import *
from modelings.modelings_roberta import *
from modelings.modelings_gpt2 import *
from modelings.modelings_lstm import *
"""
For evaluate, we use a single random seed, as
the models are trained with 5 different seeds
already.
"""
_ = random.seed(123)
_ = np.random.seed(123)
_ = torch.manual_seed(123)

#### Main evaluate script

In [139]:
results = {}
# rerunning the boxes below will only append stuffs to the results.

In [140]:
"""
The following blocks will run CEBaB benchmark in
all the combinations of the following conditions.
"""
grid = {
    "eval_split": ["test"],
    # dev,test
    "control": ["ks"],
    # baseline-random,hdims,layers
    "seed": [42],
    # 42, 66, 77
    "h_dim": [192],
    # 1,16,64,128,192
    # 1,16,64,75
    "interchange_layer" : [12],
    # 0,1; 2,4,6,8,10,12
    "class_num": [5],
    "k" : [10,100,500,1000,3000,6000,9848], 
    # 0,10,100,500,1000,3000,6000,9848,19684
    "alpha" : [1.0],
    # 0.0,1.0
    "beta" : [1.0],
    # 0.0,1.0
    "gemma" : [3.0],
    # 0.0,3.0
    "model_arch" : ["gpt2"],
    # lstm, bert-base-uncased, roberta-base, gpt2
    "lr" : ["8e-05"],
    # 8e-05; 0.001
    "counterfactual_type" : ["true"]
    # approximate,true
}

keys, values = zip(*grid.items())
permutations_dicts = [dict(zip(keys, v)) for v in itertools.product(*values)]

device = 'cuda:0'
batch_size = 32

In [141]:
for i in range(len(permutations_dicts)):
    
    eval_split=permutations_dicts[i]["eval_split"]
    seed=permutations_dicts[i]["seed"]
    class_num=permutations_dicts[i]["class_num"]
    alpha=permutations_dicts[i]["alpha"]
    beta=permutations_dicts[i]["beta"]
    gemma=permutations_dicts[i]["gemma"]
    h_dim=permutations_dicts[i]["h_dim"]
    dataset_type = f'{class_num}-way'
    control=permutations_dicts[i]["control"]
    model_arch=permutations_dicts[i]["model_arch"]
    k=permutations_dicts[i]["k"]
    interchange_layer=permutations_dicts[i]["interchange_layer"]
    lr=permutations_dicts[i]["lr"]
    counterfactual_type=permutations_dicts[i]["counterfactual_type"]
    
    if model_arch == "bert-base-uncased":
        model_path = "BERT"
        model_module = BERTForCEBaB
        explainer_module = CausalProxyModelForBERT
    elif model_arch == "roberta-base":
        model_path = "RoBERTa" 
        model_module = RoBERTaForCEBaB
        explainer_module = CausalProxyModelForRoBERTa
    elif model_arch == "gpt2":
        model_path = "gpt2"
        model_module = GPT2ForCEBaB
        explainer_module = CausalProxyModelForGPT2
    elif model_arch == "lstm":
        model_path = "lstm"
        model_module = LSTMForCEBaB
        explainer_module = CausalProxyModelForLSTM
    model_path += f"-{control}"
    grid_conditions=(
        ("eval_split", eval_split),
        ("control", control),
        ("seed", seed),
        ("h_dim", h_dim),
        ("interchange_layer", interchange_layer),
        ("class_num", class_num),
        ("k", k),
        ("alpha", alpha),
        ("beta", beta),
        ("gemma", gemma),
        ("model_arch", model_arch),
        ("lr", lr),
        ("counterfactual_type", counterfactual_type)
    )
    print("Running for this setting: ", grid_conditions)

    blackbox_model_path = f'CEBaB/{model_arch}.CEBaB.sa.'\
                          f'{class_num}-class.exclusive.seed_{seed}'
    cpm_model_path = f'../proxy_training_results/{model_path}/'\
                     f'cebab.alpha.{alpha}.beta.{beta}.gemma.{gemma}.'\
                     f'lr.{lr}.dim.{h_dim}.hightype.{model_arch}.'\
                     f'CEBaB.cls.dropout.0.1.enc.dropout.0.1.counter.type.'\
                     f'{counterfactual_type}.k.{k}.int.layer.{interchange_layer}.'\
                     f'seed_{seed}/'

    # load data from HF
    cebab = datasets.load_dataset(
        'CEBaB/CEBaB', use_auth_token=True,
        cache_dir="../train_cache/"
    )

    train, dev, test = preprocess_hf_dataset_inclusive(
        cebab, verbose=1, dataset_type=dataset_type
    )

    eval_dataset = dev if eval_split == 'dev' else test

    tf_model = model_module(
        blackbox_model_path, 
        device=device, 
        batch_size=batch_size
    )
    explainer = explainer_module(
        blackbox_model_path,
        cpm_model_path, 
        device=device, 
        batch_size=batch_size,
        intervention_h_dim=h_dim,
    )

    result_per_example, ATE, CEBaB_metrics, CEBaB_metrics_per_aspect_direction, \
    CEBaB_metrics_per_aspect, CaCE_per_aspect_direction, \
    ACaCE_per_aspect, performance_report = cebab_pipeline(
        tf_model, explainer, 
        train, eval_dataset,
        seed, k, dataset_type=dataset_type, 
        shorten_model_name=False, 
        train_setting="inclusive", 
        approximate=False if counterfactual_type == "true" else True
    )
    
    results[grid_conditions] = (
        result_per_example, ATE, CEBaB_metrics, CEBaB_metrics_per_aspect_direction, \
        CEBaB_metrics_per_aspect, CaCE_per_aspect_direction, \
        ACaCE_per_aspect, performance_report
    )


Running for this setting:  (('eval_split', 'test'), ('control', 'ks'), ('seed', 42), ('h_dim', 192), ('interchange_layer', 12), ('class_num', 5), ('k', 10), ('alpha', 1.0), ('beta', 1.0), ('gemma', 3.0), ('model_arch', 'gpt2'), ('lr', '8e-05'), ('counterfactual_type', 'true'))


Using custom data configuration CEBaB--CEBaB-ccd674d249652bd4
Reusing dataset parquet (../train_cache/CEBaB___parquet/CEBaB--CEBaB-ccd674d249652bd4/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8)


  0%|          | 0/5 [00:00<?, ?it/s]

Dropping no majority reviews: 16.6382% of train_exclusive dataset.
Dropping no majority reviews: 16.03% of train_inclusive dataset.
intervention_h_dim=192


100%|██████████| 124/124 [00:20<00:00,  6.01it/s]


...

Running for this setting:  (('eval_split', 'test'), ('control', 'ks'), ('seed', 42), ('h_dim', 192), ('interchange_layer', 12), ('class_num', 5), ('k', 9848), ('alpha', 1.0), ('beta', 1.0), ('gemma', 3.0), ('model_arch', 'gpt2'), ('lr', '8e-05'), ('counterfactual_type', 'true'))


Using custom data configuration CEBaB--CEBaB-ccd674d249652bd4
Reusing dataset parquet (../train_cache/CEBaB___parquet/CEBaB--CEBaB-ccd674d249652bd4/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8)


  0%|          | 0/5 [00:00<?, ?it/s]

Dropping no majority reviews: 16.6382% of train_exclusive dataset.
Dropping no majority reviews: 16.03% of train_inclusive dataset.
intervention_h_dim=192


100%|██████████| 124/124 [00:20<00:00,  5.92it/s]


#### Show your results

In [142]:
important_keys = [
    "eval_split",
    "control", "seed", 
    "h_dim", "interchange_layer", 
    "class_num", "k", 
    "beta", "gemma", 
    "model_arch", "lr", "counterfactual_type"
]
values = []
for k, v in results.items():
    _values = []
    for ik in important_keys:
        _values.append(dict(k)[ik])
    _values.append(v[2]["ICaCE-L2"].iloc[0])
    _values.append(v[2]["ICaCE-cosine"].iloc[0])
    _values.append(v[2]["ICaCE-normdiff"].iloc[0])
    _values.append(v[-1].iloc[0][0])
    values.append(_values)
important_keys.extend(["ICaCE-L2", "ICaCE-cosine", "ICaCE-normdiff", "macro-f1"])
df = pd.DataFrame(values, columns=important_keys)
df.sort_values(by=['seed', 'interchange_layer', 'h_dim'], ascending=True)

,eval_split,control,seed,h_dim,interchange_layer,class_num,k,beta,gemma,model_arch,lr,counterfactual_type,ICaCE-L2,ICaCE-cosine,ICaCE-normdiff,macro-f1
0,test,ks,42,192,12,5,10,1.0,3.0,gpt2,8e-05,true,0.7381,0.6406,0.4868,0.400745
1,test,ks,42,192,12,5,100,1.0,3.0,gpt2,8e-05,true,0.6527,0.5687,0.5085,0.559982
2,test,ks,42,192,12,5,500,1.0,3.0,gpt2,8e-05,true,0.6549,0.5435,0.5101,0.605898
3,test,ks,42,192,12,5,1000,1.0,3.0,gpt2,8e-05,true,0.6242,0.5548,0.5191,0.621604
4,test,ks,42,192,12,5,3000,1.0,3.0,gpt2,8e-05,true,0.6144,0.5276,0.5149,0.643678
5,test,ks,42,192,12,5,6000,1.0,3.0,gpt2,8e-05,true,0.5626,0.4995,0.4533,0.657460
6,test,ks,42,192,12,5,9848,1.0,3.0,gpt2,8e-05,true,0.4764,0.4402,0.3494,0.659706


#### Aggregating results in the way you like

In [131]:
df.groupby(['k'], as_index=False).mean()

,k,seed,h_dim,interchange_layer,class_num,beta,gemma,ICaCE-L2,ICaCE-cosine,ICaCE-normdiff,macro-f1
0,19684,61.666667,192.0,8.0,5.0,1.0,3.0,0.4602,0.3775,0.273833,0.70713


In [132]:
df.groupby(['k'], as_index=False).std()

,k,seed,h_dim,interchange_layer,class_num,beta,gemma,ICaCE-L2,ICaCE-cosine,ICaCE-normdiff,macro-f1
0,19684,17.897858,0.0,0.0,0.0,0.0,0.0,0.010159,0.007927,0.009251,0.004396


#### Save your results somewhere and load again to tabularize your results altogether

In [ ]:
output_name = input("Plase give an output file name: ")

output_directory = f'../proxy_training_results/{model_path}/'
output_filename = os.path.join(output_directory, f'{output_name}.pkl')
print("Writing to file: ", output_filename)
with open(output_filename, 'wb') as f:
    pickle.dump(results, f)